In [4]:
# Train the model for level 2 new data same model
import os
import torch
import pandas as pd
import numpy as np
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from datetime import datetime
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from params import device, SEQ_LENGTH
from model import HybridLSTM, HybridGRU
from dataset import SplitDataset
from labeling import label_mapping, label_removing
from training import training_hybrid
from testing import testing_hybrid

# Define the path
path = 'data/'
md_path = 'new_model_pth/'


LOOKBACK = SEQ_LENGTH - 1
num_epochs = 200

# Sequence data with sequence length
seq_data = np.load(f'{path}/trans_new_{SEQ_LENGTH}.npy', allow_pickle=True) # (6, 770400, 8)
seq_data = np.transpose(seq_data, (1, 0, 2))

seq_x = seq_data[:LOOKBACK, :, 1: -3].astype(np.float64) # Temporarily remove 0 (the time)
seq_y1 = seq_data[LOOKBACK, :, -2].astype(np.float64) # Predict the box
seq_y2 = seq_data[LOOKBACK, :, -1].astype(np.float64)
seq_x = np.transpose(seq_x, (1, 0, 2))

# Time lag
added_x = np.transpose(seq_data[:, :, 0], (1, 0))
datetime_array, hour_array = [], []
for row in added_x:
    datetime_row = [datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S') for date_string in row]
    diff_list = [int((datetime_row[i+1] - datetime_row[i]).seconds / 60 )for i in range(len(datetime_row)-1)]
    hour_list = [int(item.hour) for item in datetime_row[:-1]]
    datetime_array.append(diff_list)
    hour_array.append(hour_list)
    

# Concatenate the two arrays along the new axis (axis=2)
dt_arr = np.expand_dims(np.array(datetime_array), axis=2)
hr_arr = np.expand_dims(np.array(hour_array), axis=2)   
seq_x = np.concatenate((dt_arr, seq_x), axis=2)
seq_x = np.concatenate((seq_x, hr_arr), axis=2)
y_cluster = label_mapping(list(seq_y1), list(seq_y2))

(2626089, 4, 12)
[[600.         1.         4.         4.        99.        97.
  139.770142  35.697872   5.         0.      ]
 [  8.         1.         4.         4.        11.         1.
  139.770139  35.698142   5.        10.      ]
 [ 20.         1.         4.         4.        11.        12.
  139.764575  35.699903   5.        10.      ]]


In [21]:
seq_length = seq_x.shape[0]
cnt = 0
# for i in range(seq_length):
#     tmp = seq_x[i,-1,-1]
#     if int(tmp) != y_cluster[i]:
#         cnt += 1
    



In [22]:
cnt / len(y_cluster)

0.989766150347532

In [26]:
l = [96.04, 94.82, 96.01, 94.95, 96.65, 95.33]
l = [round(i-4.05,2) for i in l]
l

[91.99, 90.77, 91.96, 90.9, 92.6, 91.28]

In [35]:
seq_data = np.load(f'{path}/trans_new_{SEQ_LENGTH}.npy', allow_pickle=True)
seq_len = seq_data.shape[0]

ls = []
for i in range(seq_len):
    ls.append(seq_data[i, -2, 5])



In [36]:
ls.count(97) / seq_len

0.31700867716212205

In [37]:
l = [0.3170, 0.2898, 0.3066, 0.2775, 0.3099, 0.2811]
for i in l:
    print(1-i)

0.683
0.7101999999999999
0.6934
0.7224999999999999
0.6900999999999999
0.7189
